In [1]:
import pandas as pd
from datetime import datetime, timedelta
import praw

In [2]:
reddit = praw.Reddit(
    client_id = '7QZb12vzjMWBB1b0yQG32Q',
    client_secret = 'jVSRfTtAMaMVmzeYrbvToZQFsrhM5A',
    user_agent = 'Stock_Movement_Analysis by Background_Second725'
)

print('Reddit API connection : ', reddit.read_only)

Reddit API connection :  True


In [3]:
def scrape_subreddit(reddit, subreddit_name, start_date, end_date, post_limit=1000):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []

    for post in subreddit.new(limit=post_limit):
        post_date = datetime.fromtimestamp(post.created_utc)
        if start_date <= post_date <= end_date:
            posts.append({
                'title' : post.title,
                'id' : post.id,
                'author' : str(post.author),
                'created_utc' : post_date,
                'score' : post.score,
                'upvote_ratio' : post.upvote_ratio,
                'num_comments' : post.num_comments,
                'url' : post.url,
                'selftext' : post.selftext
            })

    return pd.DataFrame(posts)

In [4]:
def scrape_multiple_subreddits(subreddits, months=6, post_limit=1000):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30 * months)
    combined_data = pd.DataFrame()
    
    for subreddit in subreddits:
        print(f'Scaping subreddit : {subreddit} from {start_date} to {end_date}')
        subreddit_data = scrape_subreddit(
            reddit, subreddit, start_date, end_date, post_limit=post_limit
        )
        combined_data = pd.concat([combined_data, subreddit_data], ignore_index=True)

    return combined_data

In [5]:
subreddits_to_scrape = ['stocks', 'investing', 'wallstreetbets']
months_to_scrape = 6
max_posts_per_subreddit = 1000

scraped_data = scrape_multiple_subreddits(
    subreddits = subreddits_to_scrape,
    months = months_to_scrape,
    post_limit = max_posts_per_subreddit
)

Scaping subreddit : stocks from 2024-06-09 23:51:05.224600 to 2024-12-06 23:51:05.224600
Scaping subreddit : investing from 2024-06-09 23:51:05.224600 to 2024-12-06 23:51:05.224600
Scaping subreddit : wallstreetbets from 2024-06-09 23:51:05.224600 to 2024-12-06 23:51:05.224600


In [6]:
output_file = '../Data/reddit_stock_data.csv'
scraped_data.to_csv(output_file, index=False)
print(f'Scraped data saved to {output_file}')

Scraped data saved to ../Data/reddit_stock_data.csv


In [7]:
print(f'Total posts scraped : {len(scraped_data)}')

Total posts scraped : 2380
